# Stream anomaly detector POC

For real-time data monitoring, it is recommend to detect the anomaly status of the latest data point only. By continuously applying latest point detection, streaming data monitoring can be done more efficiently and accurately.

Reference - https://docs.microsoft.com/en-us/azure/cognitive-services/anomaly-detector/concepts/anomaly-detection-best-practices

To send requests to the Anomaly Detector API, we require anomaly detector access key and enpoint for the anomaly detector resource. These can be found in `Keys and Endpoint` menu item on left navigation bar.

In [ ]:
apikey = '<Your api key>'
endpoint = '<Your endpoint>'
batch_detection_endpoint = f'{endpoint}/anomalydetector/v1.1-preview/timeseries/last/detect'

# Steps to run

* Verify `apikey` and `endpoint` variables above have been updated with correct values.
* Click on `Kernel` menu.
* Click on `Restart & Run All` menu item.

In [ ]:
import requests
import json

**Below is the function that'll make calls to Anomaly Detector API's batch detection endpoint**

In [ ]:
def detect(endpoint, apikey, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': apikey}
    response = requests.post(endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

**Below is the function that gets sample data from microsoft's github repo**

In [ ]:
def get_sample_data():
    url = 'http://raw.githubusercontent.com/Azure-Samples/AnomalyDetector/master/ipython-notebook/sample.json'
    response = requests.get(url, verify=False)
    return response.text

## Result
**Below is the code that fetches sample data and calls anomaly detector endpoint.**

Anomaly detector endpoint returns anomaly information for only the last point in the sample data.

In [ ]:
sample_run_config = {
    'add_anomalous_data_point': True, #add an anamolus data point at the end of the sample data aray
    'anamalous_data': 80000000         #anamalous data point value
}

In [ ]:
# Ignore the certifcate warning in the final output.

data = get_sample_data()
json_data = json.loads(data)
json_data['sensitivity'] = 95

if sample_run_config['add_anomalous_data_point']:
    json_data['series'].append({
        'timestamp': "2018-04-17T00:00:00Z",
        'value': sample_run_config['anamalous_data']
    })

result = detect(batch_detection_endpoint, apikey, json_data)

# added last point information to final result
# so that it is easy to infer anomaly information
result['lastPoint'] = json_data['series'][len(json_data['series']) - 1]

print(json.dumps(result, indent=4))